In [1]:
from io import StringIO
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import os
import glob
from scipy.integrate import cumtrapz
from scipy.fft import fft
import numpy as np
from obspy import Trace
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from sklearn.ensemble import IsolationForest
from scipy import signal
from sklearn.linear_model import LinearRegression




lunar_cat_directory = './space_apps_2024_seismic_detection/data/lunar/training/catalogs/'
lunar_cat_file = lunar_cat_directory + 'apollo12_catalog_GradeA_final.csv'
lunar_cat = pd.read_csv(lunar_cat_file)

# load data directories
lunar_training_data_directory = './space_apps_2024_seismic_detection/data/lunar/training/data/S12_GradeA/'
mars_training_data_directory = './space_apps_2024_seismic_detection/data/mars/training/data'


lunar_training_file = lunar_training_data_directory + lunar_cat['filename'][0] + '.csv'

l_training_df = pd.read_csv(lunar_training_file)

In [2]:
def sta_lta(df, expand, sta=500, lta=5000, thr_on=3, thr_off=1):
    '''
    Dectect and mark trigger on and trigger off times of seismic anomalies using Short Term Average/Long Term Average algorithm

    Parameters:
    - df (pd.DataFrame): DataFrame containing the seismic data.
    - sta (int): Short Time Average window length (in samples).
    - lta (int): Long Time Average window length (in samples).
    - thr_on (float): Trigger on threshold (earthquake detected) in rel time.
    - thr_off (float): Trigger off threshold in rel time.

    Returns:
    - list: A list of tuples containing trigger on and off indices of df.
    '''
    rel_time = np.array(df["time_rel(sec)"])
    samp_rate = 1 / (rel_time[-1] - rel_time[-2])
    offset = int(samp_rate * 240) * expand

    cft = classic_sta_lta(df["velocity(m/s)"], int(sta * samp_rate), int(lta * samp_rate))
    on_off = np.array(trigger_onset(cft, thr_on, thr_off))

    trig_indices = []
    for i in np.arange(0, len(on_off)):
        trigger = on_off[i]
        trig_indices.append((trigger[0] - offset, trigger[1] + offset))

    return trig_indices

def regression_analysis(df):
    """
    Analyzes amplitude (absolute value of velocity) over time data by performing linear regression, returning the correlation (R²), and plotting the amplitude data with the regression line.

    Parameters:
    - df_slice (pd.DataFrame): DataFrame slice containing possible seismic event.

    Returns:
    - float: The correlation coefficient (R² value) representing the linear fit.
    """
    time = df['time_rel(sec)'].values.reshape(-1, 1)
    velocity = df['velocity(m/s)'].values
    amplitude = np.abs(velocity)
    model = LinearRegression()
    model.fit(time, amplitude)
    
    # Get the slope and correlation coefficient (R² value)
    slope = model.coef_[0]
    r_squared = model.score(time, amplitude)
    
    # Generate predicted values for the regression line on amplitude
    amplitude_pred = model.predict(time)

    plt.figure(figsize=(12, 6))
    plt.plot(time, amplitude, label='Amplitude (|Velocity|)', color='blue', alpha=0.5)
    plt.plot(time, amplitude_pred, label='Linear Regression Line', color='red', linestyle='--')
    plt.xlabel('Time (seconds)')
    plt.ylabel('Amplitude (|Velocity| in m/s)')
    plt.title('Amplitude over Time with Linear Regression Line')
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()
    print(f"Slope: {slope}")
    print(f"R² (correlation coefficient): {r_squared}")
    
    return r_squared

def is_seismic(df):
    """
    Analyzes the possible slices of seismic data by finding trigger events, slicing them, and running linear regression to determine if they represent seismic activity.
    
    Returns:
    - list: A list of indices of true positives
    """
    sta2, lta2 = 50, 200
    
    trig_inds = sta_lta(df, 1)
    new_inds = []
    for seg in trig_inds:
        new_inds += sta_lta(l_training_df.iloc[seg[0]: seg[1]], 0, sta = sta2, lta =lta2)
        
    false_positives = []
    true_positives = []
    
    for i, (start_idx, end_idx) in enumerate(new_inds):
        start_idx = max(0, start_idx)
        end_idx = min(len(df), end_idx)
        
        df_slice = df.iloc[start_idx:end_idx]
        print(f"Analyzing slice {i + 1} (from index {start_idx} to {end_idx})")
        r_squared_value = is_seismic(df_slice)
        print(f"Slice {i + 1} - Correlation Coefficient (R²): {r_squared_value}\n")
        
        if r_squared_value > 0.1:
            true_positives.append(start_idx)
        else:
            false_positives.append(start_idx)
            
    print(f"false_positives: {false_positives}")
    print(f"true_positives: {true_positives}")
    
    return true_positives



In [18]:
sta2, lta2 = 50, 200

trig_inds = sta_lta(l_training_df, 1)
new_inds = []
for seg in trig_inds:
    new_inds += sta_lta(l_training_df.iloc[seg[0]: seg[1]], 0, sta = sta2, lta =lta2)
    print(new_inds)


[]
[]
[(1324, 1916), (5384, 5940)]
[(1324, 1916), (5384, 5940), (6920, 7231), (7864, 9042)]
[(1324, 1916), (5384, 5940), (6920, 7231), (7864, 9042)]
[(1324, 1916), (5384, 5940), (6920, 7231), (7864, 9042), (8498, 9999)]
[(1324, 1916), (5384, 5940), (6920, 7231), (7864, 9042), (8498, 9999), (1323, 2074), (5942, 7239)]
[(1324, 1916), (5384, 5940), (6920, 7231), (7864, 9042), (8498, 9999), (1323, 2074), (5942, 7239)]
[(1324, 1916), (5384, 5940), (6920, 7231), (7864, 9042), (8498, 9999), (1323, 2074), (5942, 7239)]
[(1324, 1916), (5384, 5940), (6920, 7231), (7864, 9042), (8498, 9999), (1323, 2074), (5942, 7239), (1323, 3734)]
